# Capstone Project Battle of the Neighborhoods Part 1 #

## Introduction ##

For this project, I decided to examine New York City. I have a bit of a sweet tooth, as I know many others do as well, so I decided I would see which areas in New York would be best to visit to satisfy that craving. My plan is to map every candy store, ice cream stop, and chocolate shop I can find, then make clusters to map New York into sweet zones.

## The Data ##

This project will rely on venue data from Foursquare, along with the neighborhood data for New York. I'll perform a search for every type of sweet category I can find on foursquare on each neighborhood, then map all those locations. Once I have all this data, I'll make clusters to find the best areas.